## Install dependencies

In [15]:
#imports.... Run this each time after restarting the Kernel
#!pip install watson_developer_cloud
import watson_developer_cloud as watson
import json
from botocore.client import Config
import ibm_boto3


### Add Credentials

Copy paste the following snippet to next cell, and add your own set of crdentials there:

```code
credentials_os = {
    'IBM_API_KEY_ID': '',
    'IAM_SERVICE_ID': '',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': '',
}

credentials_stt = {
    "url": "",
    "username": "",
    "password": ""
}

```

In [16]:
# The code was removed by DSX for sharing.

In [17]:
client = ibm_boto3.client(
    service_name = 's3', 
    ibm_api_key_id = credentials_os['IBM_API_KEY_ID'],
    ibm_auth_endpoint = credentials_os['IBM_AUTH_ENDPOINT'],
    config = Config(signature_version = 'oauth'),
    endpoint_url = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'
)


## Speect to Text 

Following cell has two methods:
 - `get_transcript()` calls speech to text enpoint and generates a text transcript for you for a sample audio file.
 - `analyze_sample()` gets the sample object from cloud storage, calls get_transcript to fetch the tranccript, and saves your transcript in cloud storage as `<file_name>_text.json`.

In [18]:
#STT

import json
import io
from os.path import join, dirname
from watson_developer_cloud import SpeechToTextV1

speech_to_text = SpeechToTextV1(
    username = credentials_stt['username'],
    password = credentials_stt['password'],
    x_watson_learning_opt_out=False
)


#get transcript Very basic one
def get_transcript(audio):
    transcript = json.dumps(speech_to_text.recognize(audio, content_type='audio/ogg', timestamps=True,
        word_confidence=True), indent=2)
    return transcript

def analyze_sample(sample):
    streaming_body = client.get_object(Bucket='watsoncallcenterthink18c8855d42de924dc38ccd02f3a8a50d7f', Key=sample)['Body']
    text = get_transcript(streaming_body.read())
    client.put_object(Bucket='watsoncallcenterthink18c8855d42de924dc38ccd02f3a8a50d7f', Key = sample.split('.')[0] + '_text.json', Body = text)
    return text


## Analyze the selected files


In [19]:
file_list = ['sample1-addresschange-positive.ogg',
             'sample2-address-negative.ogg',
             'sample3-shirt-return-weather-chitchat.ogg',
             'sample4-angryblender-sportschitchat-recovery.ogg',
             'sample5-calibration-toneandcontext.ogg']


In [34]:
transcript = analyze_sample(file_list[0])

for result in json.loads(transcript)['results']:
    print(result['alternatives'][0]['transcript'], result['alternatives'][0]['confidence'])

(u'good morning ', 0.995)
(u"can you give me some help I'd like to change my address please ", 0.999)
(u'my name is Ryan Smith ', 0.972)
(u'I am from Sacramento California ', 0.969)
(u"that's right ", 0.996)
(u'my phone number is five five five one two one two ', 0.839)
(u"yes that's me ", 0.997)
(u'my old address is number one two three oak street ', 0.965)
(u'my new address is five six seven pine street ', 0.865)
(u'yes and the zip is nine zero two one zero ', 0.921)
(u"yep that's right ", 0.89)
(u'now the phone number stays the same ', 0.808)
(u"that's right I would like to keep all the options of said no other changes the only thing that I want to change is the address ", 0.944)
(u"yes that's right ", 0.995)
(u'yep ', 0.583)
(u'very good yes thank you so much for help ', 0.921)
(u'it ', 0.419)
(u'thanks have a good day bye bye ', 0.994)


In [36]:
transcript = analyze_sample(file_list[1])

for result in json.loads(transcript)['results']:
    print(result['alternatives'][0]['transcript'], result['alternatives'][0]['confidence'])

(u'I am ', 0.506)
(u'yeah I want to change my address ', 0.895)
(u'naw ', 0.276)
(u"you know this is the third time this week I've had to call in to change a setting for some reason you keep sending my mail to the old address I'm getting pretty frustrated ", 0.833)
(u'so are ', 0.812)
(u"can you just get it changed and hopefully you guys don't screwed up again ", 0.775)
(u'okay ', 0.992)
(u"yeah it's Bob Smith in Sacramento ", 0.957)
(u'and my old addresses one twenty three oak street ', 0.849)
(u'and my new address is four five six pine street ', 0.826)
(u"yep that's right ", 0.922)
(u'and is it is nine zero two one zero ', 0.892)
(u"and I gotta tell you I'm getting pretty frustrated having to call you guys back this is %HESITATION it's kind of pissed me off ", 0.761)
(u"all right Sir you're certain I'm not happy changed again ", 0.704)
(u'all right ', 0.553)
(u'yep ', 0.517)
(u"that's correct ", 0.943)
(u'all right ', 0.603)
(u'okay bye bye ', 0.833)


In [ ]:
### analyze more samples and display results in better format 